In [7]:
from liveBot import *
%matplotlib inline


In [63]:
class LiveBot:

    # funciones necesarias para el manejo del tiempo
    from datetime import datetime, timedelta
    from time import sleep

    def __init__(self, pair, period, strategy, weight=1):
            
        self.pair = pair
        self.period = period
        self.strategy = strategy
        self.weight = weight
        # en estrategía de ML, porcentaje de datos que se utilizaran 
        # en el conjunto de datos de entrenamiento
        self.per = 0.95
        # en esta lista deben guardarse los nombres de todas las estrategias
        # de machine learning en strategy, para discriminar en base a estas
        # la cantindad de datos a traer 
        self.ml_strategies = ["ml_logreg", "ml_randfor", "ml_knn"]
        self.ml_strategy = False
        self.have_coin = False
        self.delta = timedelta(seconds = self.period)    
        # máximo delay aceptado entre el tiempo actual y el del último dato
        self.delta2 = timedelta(seconds = 200)
        # desfase para en la mayoría de los casos no entrar al while
        self.delta3 = timedelta(seconds = 90)
        
        print "Creado live Bot con las siguientes características:\n"
        print "\tpair: {0}, period: {1}, strategy: {2}, weight: {3}\n".format(self.pair, self.period,
                                                                             self.strategy, self.weight)
     
    def prepareData(self):
        """Trae el dataframe y el objeto poloniex correspondiente"""
        # definiendo end como la hora local actual
        self.tf = datetime.now()
        # convirtiendola a formato unix time (es equivalente a UTC)
        self.end = string2ts(self.tf.strftime('%Y-%m-%d %H:%M:%S'))
        #start = end 

        # definiendo el tiempo inicial de la consulta 
        self.to = self.tf-delta*self.len_data
        self.start = string2ts(self.to.strftime('%Y-%m-%d %H:%M:%S'))

        # trayendo y preparando datos
        self.df, self.polo = prepareLiveData(pair=self.pair, start=self.start, end=self.end, period=int(self.period))
    
    def startBot(self):
        """Trae los datos (dataFrame y objeto polo) siempre y cuando el delay entre el tiempo actual
        y el últimio dato en el dataFrame no sea mayor a 200 segundos (delta2).
        """
        self.len_data = 0
        # dinero inicial con el que empieza el paperBot
        balance = []
        # definiendo el tiempo inicial de la consulta 

        if self.strategy in self.ml_strategies:
            # para estrategias de machine learning se tomarán los últimos
            # 7000 datos
            self.len_data = 7000
            self.ml_strategy = True
        else: 
            # para estrategias diferentes a las de ML se toman los últimos
            # 50 datos
            self.len_data = 40

            
def timeSincronizator(self):
    """Sincroniza el tiempo del bot"""
    prepareData()
    tt = self.df.index[-1]
    de = self.tf - tt

    # sincronizando tiempo del bot
    if de > self.delta2:
        # mientras la diferencia entre el la hora del último precio de cierre y
        # y la hora actual sea mayor a 120s
        while de > self.delta2:
            prepareData()
            tt = df.index[-1]
            de = self.tf - tt
            sys.stdout.write("\rSincronizando bot (delay máximo aceptado 200s, actual %ss). Esperando cierre de las %s. Último dato de cierre a las %s"%(str((de).seconds), str(tt+self.delta), str(tt)))
            sys.stdout.flush()
            sleep(5)

In [64]:
firstBot = LiveBot("BTC_DGB", "300", "ml_knn")
firstBot.delta

NameError: global name 'timedelta' is not defined

In [37]:
class MultiLiveBot():
    
    def __init__(self):
        """Clase que administra el funcionamiento de varios bots a la vez
        bots = lista de python con los bots a administrar 
        """
        self.bot_loader()
        self.n_bots = len(self.bots)
    
    def bot_loader(self):
        """Carga los datos para crear los bots desde el archivo
        multi_bot.inp, cuyo formato por línea es:
        par, %, estrategia, periodo
        """
        bots = []
        
        # abriendo archivo como lista
        multi_file = open("multi_bot.inp").readlines()
        
        for line in multi_file:
            if line.strip("\n").strip(" ") != "":
                pair = line.split(",")[0].strip(" ")
                weight = line.split(",")[1].strip(" ")
                strategy = line.split(",")[2].strip(" ")
                period = line.split(",")[3].strip("\n").strip(" ")
                bots.append({"pair":pair, "weight":weight, "strategy":strategy, "period":period})
        
        self.bots = bots
        
        # Lista con todos los pares del MultiBot
        pairs = []
        for bot in bots:
            pairs.append(bot["pair"])
        
        self.pairs = pairs
    
    def fundTrackerConstructor(self):
        """Crea archivo fundTracker con su header."""
        
        bots = ""
        for pair in self.pairs:
            bots += pair+","
            
        with open("fundTracker.inp","w") as tracker_file:
            tracker_file.write("Fecha,"+bots.strip(",")+"\n")
        
    def fundTracker(self, fecha, balances):
        """Escribe la distribución de los fondos administrados por
        cada bot en el archivo fundTracker.inp, cuyo formato por linea es:
        Fecha, balance par_1, balance par_2, ..., balance par_n
        
        :param fecha: Fecha de la linea de balance a escribir
        :type fecha: string con formato YYYY-mm-dd hh:mm:ss
        
        :param balances: contiene el balance actual en cada par
        :type balances: lista de python. Los fondos en la lista deben
        seguir el orden de self.pairs"""
        
        # Se agrega linea al final del archivo con el último dato de los balances
        with open("fundTracker.inp","a") as tracker_file:
            tracker_file.write(fecha+","+",".join(balances)+"\n")
            
    
    def fundTrackerReader(self):
        """Lee los fondos administrados por cada bot. Esto es,
        lee el archivo fundTracker.inp"""
        
        # Se abre el archivo fundTracker.inp como lista 
        tracker_file = open("fundTracker.inp").readlines()
        
        # Agregamos todo menos el header
        tracker_file = tracker_file[1:]
        
        
        return tracker_file
            

In [38]:
administrador = MultiLiveBot()
administrador.fundTrackerConstructor()
administrador.fundTracker("2018-01-29 17:09:00",["0.14","0.56","0.76"])

In [10]:
administrador.bots

[{'pair': 'BTC_DGB', 'period': '300', 'strategy': 'ml_knn', 'weight': '0.4'},
 {'pair': 'BTC_BCH',
  'period': '300',
  'strategy': 'ml_logreg',
  'weight': '0.4'},
 {'pair': 'BTC_ETH', 'period': '300', 'strategy': 'ml_knn', 'weight': '0.2'}]

In [43]:
a = open("fundTracker.inp").readlines()
print administrador.pairs
print a[1:]

['BTC_DGB', 'BTC_BCH', 'BTC_ETH']
['2018-01-29 17:09:00,0.14,0.56,0.76\n']


In [34]:
administrador.pairs

['BTC_DGB', 'BTC_BCH', 'BTC_ETH']